In [8]:
from __future__ import absolute_import
from __future__ import print_function

import numpy as np
import pandas as pd
import math
import ast

import h5py
import sys
np.random.seed(1337)  # for reproducibility

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [4]:

def load_data(path, train=True):
    df = pd.read_csv(path)
    X = df.values.copy()
    if train:
        np.random.shuffle(X)  
        X, labels = X[:, 1:-1].astype(np.float32), X[:, -1]
        return X, labels
    else:
        X, ids = X[:, 1:].astype(np.float32), X[:, 0].astype(str)
        return X, ids


def preprocess_data(X, scaler=None):
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(X)
    X = scaler.transform(X)
    return X, scaler


def preprocess_labels(labels, encoder=None, categorical=True):
    if not encoder:
        encoder = LabelEncoder()
        encoder.fit(labels)
    y = encoder.transform(labels).astype(np.int32)
    if categorical:
        y = np_utils.to_categorical(y)
    return y, encoder


def make_submission(y_prob, ids, encoder, fname):
    with open(fname, 'w') as f:
        f.write('id,')
        f.write(','.join([str(i) for i in encoder.classes_]))
        f.write('\n')
        for i, probs in zip(ids, y_prob):
            probas = ','.join([i] + [str(p) for p in probs.tolist()])
            f.write(probas)
            f.write('\n')
    print("Wrote submission to file {}.".format(fname))

In [5]:
print("Loading data...")
X, labels = load_data('otto/train.csv', train=True)
X, scaler = preprocess_data(X)
y, encoder = preprocess_labels(labels)

X_test, ids = load_data('otto/test.csv', train=False)
X_test, _ = preprocess_data(X_test, scaler)

nb_classes = y.shape[1]
print(nb_classes, 'classes')

dims = X.shape[1]
print(dims, 'dims')



Loading data...
9 classes
93 dims
Building model...


In [6]:
print("Building model...")
model = Sequential()
model.add(Dense(dims, 512, init='glorot_uniform'))
model.add(PReLU((512,)))
model.add(BatchNormalization((512,)))
model.add(Dropout(0.5))

model.add(Dense(512, 512, init='glorot_uniform'))
model.add(PReLU((512,)))
model.add(BatchNormalization((512,)))
model.add(Dropout(0.5))

model.add(Dense(512, 512, init='glorot_uniform'))
model.add(PReLU((512,)))
model.add(BatchNormalization((512,)))
model.add(Dropout(0.5))

model.add(Dense(512, nb_classes, init='glorot_uniform'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam")


Building model...
Training model...
Train on 52596 samples, validate on 9282 samples
Epoch 0
52596/52596 [==============================] - 32s - loss: 1.0228 - val_loss: 0.7169
Epoch 1
52596/52596 [==============================] - 34s - loss: 0.6735 - val_loss: 0.6388
Epoch 2
52596/52596 [==============================] - 33s - loss: 0.6259 - val_loss: 0.6030
Epoch 3
52596/52596 [==============================] - 31s - loss: 0.5979 - val_loss: 0.5861
Epoch 4
52596/52596 [==============================] - 33s - loss: 0.5804 - val_loss: 0.5689
Epoch 5
52596/52596 [==============================] - 35s - loss: 0.5658 - val_loss: 0.5588
Epoch 6
52596/52596 [==============================] - 35s - loss: 0.5532 - val_loss: 0.5479
Epoch 7
52596/52596 [==============================] - 34s - loss: 0.5455 - val_loss: 0.5419
Epoch 8
52596/52596 [==============================] - 32s - loss: 0.5347 - val_loss: 0.5432
Epoch 9
52596/52596 [==============================] - 34s - loss: 0.5252 - va

In [7]:

print("Training model...")

model.fit(X, y, nb_epoch=20, batch_size=128, validation_split=0.15,show_accuracy=True)


Training model...
Train on 52596 samples, validate on 9282 samples
Epoch 0
52596/52596 [==============================] - 33s - loss: 0.4744 - acc: 0.8131 - val_loss: 0.5061 - val_acc: 0.8042
Epoch 1
52596/52596 [==============================] - 32s - loss: 0.4694 - acc: 0.8160 - val_loss: 0.5042 - val_acc: 0.8013
Epoch 2
52596/52596 [==============================] - 31s - loss: 0.4676 - acc: 0.8150 - val_loss: 0.5081 - val_acc: 0.8020
Epoch 3
52596/52596 [==============================] - 34s - loss: 0.4617 - acc: 0.8169 - val_loss: 0.5094 - val_acc: 0.8065
Epoch 4
52596/52596 [==============================] - 39s - loss: 0.4596 - acc: 0.8180 - val_loss: 0.5142 - val_acc: 0.8010
Epoch 5
52596/52596 [==============================] - 40s - loss: 0.4579 - acc: 0.8187 - val_loss: 0.5132 - val_acc: 0.8039
Epoch 6
52596/52596 [==============================] - 33s - loss: 0.4569 - acc: 0.8180 - val_loss: 0.5053 - val_acc: 0.8031
Epoch 7
52596/52596 [==============================] - 33s

NameError: name 'h5py' is not defined

In [12]:
model.save_weights("otto1.hdf5",overwrite=True)
f = h5py.File('otto1.hdf5','r')
print("Generating submission...")

proba = model.predict_proba(X_test)
make_submission(proba, ids, encoder, fname='keras-otto.csv')

Generating submission...
144368/144368 [==============================] - 12s    
Wrote submission to file keras-otto.csv.
